# Analysis: Piece Counts by Theme

Wokflow: 4   
Goal: Run ANOVA and welch tests to investigate if average number of pieces are different by theme.   

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
%load_ext autoreload
%autoreload 2         # reload custom py file eveytime a new cell is run

import brickset_helper as bh

## Import Data

In [4]:
SET_FILE = 'data/brickset_set_clean.csv'

In [5]:
set_df = pd.read_csv(SET_FILE)
set_df.head()
set_df.info()

,set_no,name,url,theme_group,theme,subtheme,year,piece_cnt,minifig_cnt,inventory_url,minifig_url,price_store,price_new,price_used,rating_value,rating_votes,tags,set_type,packaging
0,722-1,"Universal Building Set, 7+",/sets/722-1/Universal-Building-Set-7,Basic,Basic,NaN,1980.0,301.0,0.0,NaN,NaN,20.00,200.0,78.0,4.4,5.0,"4X4, Aircraft, Articulated Lorry, Boat, Car, C...",Normal,NaN
1,733-1,"Universal Building Set, 7+",/sets/733-1/Universal-Building-Set-7,Basic,Basic,NaN,1980.0,533.0,0.0,NaN,NaN,39.00,566.0,109.0,4.5,2.0,"Articulated Lorry, Base, Baseplate, Crane, For...",Normal,NaN
2,744-1,"Universal Building Set with Motor, 7+",/sets/744-1/Universal-Building-Set-with-Motor-7,Basic,Basic,NaN,1980.0,537.0,0.0,NaN,NaN,59.00,252.0,97.0,5.0,2.0,"8 Wheel Vehicle, Crane, Electric Motor, Mobile...",Normal,NaN
3,820-1,Red Plates Parts Pack,/sets/820-1/Red-Plates-Parts-Pack,Basic,Basic,Supplementaries,1980.0,34.0,0.0,NaN,NaN,4.75,NaN,NaN,NaN,NaN,NaN,Normal,NaN
4,822-1,Blue Plates Parts Pack,/sets/822-1/Blue-Plates-Parts-Pack,Basic,Basic,Supplementaries,1980.0,34.0,0.0,NaN,NaN,4.75,29.0,5.0,NaN,NaN,NaN,Normal,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13754 entries, 0 to 13753
Data columns (total 19 columns):
set_no           13754 non-null object
name             13754 non-null object
url              13754 non-null object
theme_group      13754 non-null object
theme            13754 non-null object
subtheme         10040 non-null object
year             13754 non-null float64
piece_cnt        10858 non-null float64
minifig_cnt      13754 non-null float64
inventory_url    6958 non-null object
minifig_url      6264 non-null object
price_store      8378 non-null float64
price_new        8516 non-null float64
price_used       7015 non-null float64
rating_value     6813 non-null float64
rating_votes     6813 non-null float64
tags             10265 non-null object
set_type         13753 non-null object
packaging        8919 non-null object
dtypes: float64(8), object(11)
memory usage: 2.0+ MB


In [7]:
remove_theme_ls = ['Gear', 'Duplo', 'Collectable Minifigures', 'Promotional', 'Bionicle', 'Creator', 'Basic',
                  'Service Packs', 'Technic', 'Friends', 'Books', 'Education', 'Seasonal']
set_df = set_df.loc[set_df['theme'].isin(remove_theme_ls)==False]
set_df['theme'].value_counts()

Star Wars                            662
Town                                 594
City                                 547
Space                                294
Castle                               277
Ninjago                              242
Racers                               225
Dacta                                146
Legends of Chima                     141
Bulk Bricks                          141
Sports                               131
Explore                              127
Miscellaneous                        126
Trains                               115
HERO Factory                         106
Marvel Super Heroes                  104
Nexo Knights                         101
DC Comics Super Heroes                93
Fabuland                              92
Mixels                                90
Clikits                               85
Belville                              82
Pirates                               81
Freestyle                             75
Harry Potter    

In [8]:
remove_theme_ls = ['Racers', 'Dacta', 'Bulk Bricks', 'Sports', 'Explore', 'Miscellaneous', 'HERO Factory', 
                   'Fabuland', 'Mixels', 'Clikits', 'Belville', 'Freestyle', 'BrickHeadz', 'Scala', 
                   'Dimensions', 'Bricks and More']
set_df = set_df.loc[set_df['theme'].isin(remove_theme_ls)==False]
set_df['theme'].value_counts()

Star Wars                            662
Town                                 594
City                                 547
Space                                294
Castle                               277
Ninjago                              242
Legends of Chima                     141
Trains                               115
Marvel Super Heroes                  104
Nexo Knights                         101
DC Comics Super Heroes                93
Pirates                               81
Harry Potter                          75
Adventurers                           72
Juniors                               67
Mindstorms                            64
Creator Expert                        63
Studios                               55
Primo                                 52
Classic                               51
Games                                 49
Architecture                          46
The LEGO Batman Movie                 44
Exo-Force                             44
Make and Create 

## Export to csv

In [6]:
save_path = r'data/file_clean.csv'
file_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

NameError: name 'file_df' is not defined